In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load the data
df = pd.read_csv('labeled-data.csv')

/Users/ulasayyilmaz/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Exclude examples labeled 'N' or 'Done'
df_filtered = df[(df['label'] != 'N') & (df['label'] != 'Done')]

# Extract features and labels
X = df_filtered.iloc[:, 2:9].values  # assuming your features start from the 3rd column
y = df_filtered['label'].values

# Specify the order of your classes
class_order = ['L', 'R']  # Replace with your actual class names

# Initialize and fit the LabelEncoder with specified classes
label_encoder = LabelEncoder()
label_encoder.fit(class_order)

LabelEncoder()

In [3]:
# Transform labels to encoded format
y_encoded = label_encoder.transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [4]:
# Reshape data for LSTM input (assuming time steps = 1)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [5]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20/20 [==============================] - 1s 18ms/step - loss: 0.6667 - accuracy: 0.6970 - val_loss: 0.6622 - val_accuracy: 0.6624
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6301 - accuracy: 0.7002 - val_loss: 0.6546 - val_accuracy: 0.6624
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 0.7002 - val_loss: 0.6561 - val_accuracy: 0.6624
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6012 - accuracy: 0.7002 - val_loss: 0.6533 - val_accuracy: 0.6624
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5985 - accuracy: 0.7002 - val_loss: 0.6501 - val_accuracy: 0.6624
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5954 - accuracy: 0.7002 - val_loss: 0.6507 - val_accuracy: 0.6624
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5938 - accuracy: 0.7002 - val_loss: 0.6474 - val_accuracy: 0.6624
Epoch 8/10
20/20 [=

In [8]:
# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Test Accuracy: {accuracy}')

7/7 [==============================] - 0s 1ms/step - loss: 0.6424 - accuracy: 0.6378
Test Accuracy: 0.6377550959587097
